## 110-2 Operations Research Case 2

Algorithm 2: based on Latest Start Time, with some optimizations

Add output function

## Loading data

In [1]:
import pandas as pd 
import numpy as np

In [2]:
# !git clone https://github.com/DanielStutzbach/heapdict

In [3]:
from heapdict import heapdict
# see https://github.com/DanielStutzbach/heapdict
# or directly !pip install HeapDict
Q = heapdict()

In [8]:
datadir = './data'
instances = []
for i in range(5):
    name = f'instance_{i+1}.csv'
    fullpath = datadir+'/'+name
    instances.append(pd.read_csv(fullpath))

## Data Structures

In [9]:
# job structure 
class Job:
    '''structure for 1 job '''
    def __init__(self, row):
        '''input := df.iloc[idx, :]'''
        self.id = row['Job ID']
        self.due = row['Due Time']
        self.next_op = 0 # True as complete, False as not yet processed
        self.stage_pt = [row['Stage-1 Processing Time'], row['Stage-2 Processing Time']]
        mfor1 = list(map(int, row['Stage-1 Machines'].split(',')))
        if row['Stage-2 Machines'] is not np.nan:
            mfor2 = list(map(int, row['Stage-2 Machines'].split(',')))
        else: mfor2 = [] 
        self.stage_mach = [mfor1, mfor2]
        self.assign_mach = [None for _ in range(2)]
        self.start_time = [-1 for _ in range(2)]
        self.end_time = [-1 for _ in range(2)]
    
    def __repr__(self):
        return f'\
          * Job id: {self.id}\n\
          * Due time:{self.due}\n\
          stage 1: {self.assign_mach[0]}\n\
                   {self.stage_pt[0]}, {self.stage_mach[0]}\n\
          stage 2: {self.assign_mach[1]}\n\
                   {self.stage_pt[1]}, {self.stage_mach[1]}'
    __str__ = __repr__
        

In [10]:
class Jobs:
    '''structure for multiple jobs' management'''
    def __init__(self, n):
        self.completion_times = np.zeros(n)
        self.tardiness = np.zeros(n)
        self.is_completed = np.full(n, False)
        self.residual_times = np.zeros(n)
        self.jobs = []
        
    def get_RRDD(self):
        if getattr(self, 'RRDD', None) is None:
            self.RRDD = self.due_dates - np.min(self.due_dates)
        return self.RRDD # static

    def get_LST(self):
        '''latest start time'''
        self.LST = self.due_dates - self.residual_times
        return self.LST
    
    def add_jobs(self, data):
        self.due_dates = data['Due Time'].to_numpy()
        for i in range(len(data)):
            row = data.iloc[i, :]
            jobi = Job(row)
            self.residual_times[i] = sum(jobi.stage_pt)
            self.jobs.append(jobi)
            
    
    def assign(self, job_name, mach, st):
        '''job_name = (2, 0) means job 3 and op 1
        note that job and op is 0-indexed as well as machines
        op
        '''
        
        i = 0 
        jobidx, op = job_name 
        job = self.jobs[jobidx]
        J.completion_times[jobidx] = st + job.stage_pt[op]
        J.residual_times[jobidx] -= job.stage_pt[op]
        job.assign_mach[op] = mach
        job.start_time[op] = st
        job.end_time[op] = J.completion_times[jobidx]
        job.next_op = op+1
        if op == 1:
            self.is_completed[jobidx] = True

In [11]:
class Machines:
    def __init__(self, df):
        '''pass the stage1, stage2 machine lists'''
        mfor1 = df['Stage-1 Machines'].values.tolist()
        mfor2 = df['Stage-2 Machines'].values.tolist()
        mfor1 = [list(map(int, x.split(','))) for x in mfor1]
        mfor2 = [list(map(int, x.split(','))) for x in mfor2 if x is not np.nan]
        mfor1 = sum(mfor1, [])
        mfor2 = sum(mfor2, [])
        self.number = max(max(mfor1), max(mfor2))
        self.versatile = [mfor1.count(i+1) + mfor2.count(i+1) for i in range(self.number)]
        self.schedule = [[] for _ in range(self.number)]
        self.span = [[] for _ in range(self.number)]
        self.fintime = [0 for _ in range(self.number)]
        
    def _schedule(self, mach, job_name, st, proc_time):
        '''mach is 0-indexed'''
        display_name = tuple([x+1 for x in job_name])
        self.schedule[mach].append(f'{display_name} {proc_time:.2f}') 
        self.span[mach].append(proc_time)
        self.fintime[mach] = st + proc_time
    
    def add_idle(self, mach, idle_time):
        self.schedule[mach].append(f'idle {idle_time:.2f}') 
        self.span[mach].append(idle_time)
        self.fintime[mach] += idle_time
        

## Which instance to test?

In [52]:
INSTANCEIDX = 4
df = instances[INSTANCEIDX]

## Preprocessing 
Usage:
1. Read the dataframe into `Machines()` as `M`.
2. Init by giving the length of jobs to `Jobs()` as `J`.
    Initialize it by calling `add_jobs()`.
3. Call our heuristic algorithm. 
4. Get the result from `M, J`. No need to return them. 

In [53]:
M = Machines(df)
J = Jobs(len(df))
J.add_jobs(df)
# print(J.residual_times)
# print(J.get_LST())

## Algorithm

#### Warnings:

1. Re-run the code from **preprcoessing section** otherwise the data stuctures will keep accumulating repetitive datas. 

2. All the indexing is 0-indexed for coding convenience, but when storing back to `M.schedule` for displaying purpose, it is changed into 1-indexed. 


#### Steps:

1. ```LST = self.due_dates - self.residual_times ```

    `self.due_dates[j]` is the deadline for job ${j}$.
    `residual_times[j]` is the remaining processing times for job ${j}$.
    `LST[j]` is very similar to the idea of slackness/remaining time for you to procrastinate. 
    The less is LST, the sooner this job is to be scheduled (otherwise it's too late for it to catch up the deadline).
    


2. Jobs are put into a priority queue ${Q}$, which are ordered by their associated **LST**.


3. ```Extract_min()``` from ${Q}$ as `curr_job`, and get its corresponding operation.
    That is, if curr_job's first operation is not done yet, do first op;
    otherwise do second op. This value of which op to execute is stored in `job.next_op` attribute.

4. Calculate the best machine to schedule `curr_job`'s `curr_op`.
    The idea is to get its associated subset of machines, 
    and then order them by (1) their current finished times (for the sake of makespan), and then 
    (2) their `versatility`. Versatility is a static vector, `M.versatile[m]` specifies **the number of operations machine ${m}$ is capable of executing.** 
    To sum up, the best machine `curr_machine` is computed by: 
    ```curr_machine = min(avail_machines_idx, key = lambda m: (M.fintime[m], M.versatile[m], m))```
   Then put ```curr_job``` onto this machine.


5. To schedule or not to schedule, that is a question.
    
    **NEW FEATURE**
    Decide if this job is `PERMIT`: (1) if its current completion time plus current op's processing time is already tardy, associate it with a very big `LST`, set `PERMIT = False`. This seems to work on instance 3;
    (2) if its scheduling requires an idle time that is too large, postpone it temporarily, associate it with `LST`+0.5, set `PERMIT = False`.
    In both non-permitable cases, increment `failures` to record how many times the job is popped from ${Q}$ but fails to be scheduled. If > 2, we schedule it no matter waht. 
   
   If `PERMIT`, update values, especially update the completion_times, residual_times and **LST**,
   
   else(`not PERMIT`), push the operation back to ${Q}$ with the new `LST`. 
   
   
   For debugging purpose, I add time of idleness into M.schedule so that it is easier to read the schedule, calculate times and check feasibility. 

6. Check if all job operations are scheduled, if yes, stop the algorithm, if no, continue the iteration (go back to step 3).


#### Result:

1. `M.fintime` is the finishing time of machines, you can get the makespan by `max(M.fintime)`.
2. `M.schedule` is the schedule of machines, similar to gantt chart. 
3. `J.completion_times` is the completion times for all jobs. Comparing it with `J.due_dates` using
   ```tardies = list(np.where(J.completion_times > J.due_dates)[0])```
   gives you the tardy jobs (it's 0-indexed!!!). Turn it to 1-indexed by 
   ```[x+1 for x in tardies]```. 
   

In [54]:
# Priority Queue structure 
from heapq import heappush, heappop, heapify
Jobs_keys = [(key, job_index) for job_index, key in enumerate(J.get_LST())]
Q = Jobs_keys[:]
heapify(Q)  # sort first by slack, then by index, so it is breaking ties with smallest index 
Q[0]
# curr_job = heappop(Q)
# curr_job

(1.7763568394002505e-15, 19)

## Algorithm

In [76]:
# while not all operations in all jobs are scheduled

def heuristic(J, M): 
    # best_makepsan = sum(job_processing_time) for all jobs / |M|
    TOLRATIO = 0.3
    best_makespan = sum(job.stage_pt[0]+job.stage_pt[1] for job in J.jobs)/M.number
    tolerance = best_makespan * TOLRATIO  # tolerance for idle time
    # tolerance is a hyper-parameter; if idle > tolerance, do not schedule the curr op in the current epoch. 
    print(f'[INFO] {len(J.jobs)} jobs, {M.number} machines')
    print(f'[INFO] Tolerance: {tolerance:.2f}')
    # step 1. call get_LST()
    Q = [(lst, job_index) for job_index, lst in enumerate(J.get_LST())]
    # step 2. make priority queue 
    heapify(Q)
    fails = [0 for _ in range(len(J.jobs))]
    # step 6. check if all jobs are completed 
    epoch = 0
    while not np.all(J.is_completed):
        epoch += 1
        
        
        PERMIT = True
        # step 3. extract_min() to get the job with minimal LST and its other attributes
        _, curr_job_index = heappop(Q)
        curr_job = J.jobs[curr_job_index]
        curr_op = curr_job.next_op
        
        op_proc_time = curr_job.stage_pt[curr_op]
        job_name = (curr_job_index, curr_op)
         
        avail_machines_idx = [x-1 for x in curr_job.stage_mach[curr_op]]
        # if curr_job has no second operation 
        if op_proc_time <= 0: 
            J.assign(job_name = job_name, 
                    mach = -1,
                    st = curr_job.end_time[curr_op-1]) 
            # note that it's only possible for second operation to have proc time = 0
            # so this doesn't trigger index error
            continue 
        # step 4. calculate the best machine
        curr_machine = min(avail_machines_idx, key = lambda x: (M.fintime[x], M.versatile[x], x))
        
        # ARE THERE REASONS TO POSTPONE THE CURR OP?
        if J.completion_times[curr_job_index] + op_proc_time > J.due_dates[curr_job_index] and fails[curr_job_index] < 2:
            print(f'[INFO] Job {curr_job_index+1} will be tardy even if scheduled, queue last.')
            curr_new_value = float('inf')
            PERMIT = False
        
        # ARE THERE REASONS TO POSTPONE THE CURR OP?
        elif M.fintime[curr_machine] < J.completion_times[curr_job_index]:
            idle = J.completion_times[curr_job_index] - M.fintime[curr_machine]
            if idle > tolerance and curr_op == 1 and fails[curr_job_index] < 2:
                print(f'[INFO] Job {curr_job_index+1} op {curr_op} has idle {idle:.2f}, postpone it.')
                PERMIT = False
                if Q:
                    curr_new_value = Q[0][0] + 0.2
                else:
                    curr_new_value = 0 # the last one 
            else:
                M.add_idle( 
                mach = curr_machine, 
                idle_time = idle)
         
        if PERMIT:
            J.assign(job_name = job_name, 
                mach = curr_machine, 
                 st = M.fintime[curr_machine]
                ) 
            M._schedule(job_name = job_name, 
               mach = curr_machine, 
                proc_time = op_proc_time,
               st = M.fintime[curr_machine])
            curr_new_value = J.get_LST()[curr_job_index]
        else: 
            fails[curr_job_index] += 1
        # print(f'{epoch} Fails Count:', fails)
        # update the LST value and push it back to Q if the job has its second operation that hasn't been done
        if curr_op == 0 or not PERMIT:
            heappush(Q, (curr_new_value, curr_job_index))
            # it maintains the heap invariant, no need to heapify

In [77]:
heuristic(J, M)
J.is_completed

[INFO] 20 jobs, 9 machines
[INFO] Tolerance: 5.55


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [78]:
M.span

[[6.4, 3.1, 0.5, 5.5, 0.2, 2.6, 1.0999999999999979, 0.6],
 [6.8, 6.6, 6.5, 0.4],
 [9.6, 6.2, 0.9],
 [4.8, 3.5, 4.3, 4.9],
 [9.6, 5.0, 4.8],
 [7.6, 1.0, 3.5, 6.9],
 [9.0, 8.7, 0.6],
 [5.3, 1.7, 0.5, 5.8, 6.6],
 [8.9, 1.2, 5.4, 2.6, 5.0]]

In [79]:
J.completion_times

array([23.1,  9.6, 10.1,  8.9,  8.6, 14.6, 15.7, 16.7, 20.3, 18.3, 18.1,
       13.3, 12.6, 17.7, 20. , 18.3, 19. ,  7. , 10. , 13.4])

In [80]:
np.where(J.completion_times > J.due_dates)

(array([ 0,  2,  5,  6,  7, 10, 12, 14, 15, 16]),)

In [81]:
Tardy_jobs = list(np.where(J.completion_times > J.due_dates)[0])
Makespan = max(M.fintime)
print(f'Instance {i+1}:')
print('First objective (# tardy):', len(Tardy_jobs), Tardy_jobs)
print('Second objective (makespan):', Makespan)

Instance 5:
First objective (# tardy): 10 [0, 2, 5, 6, 7, 10, 12, 14, 15, 16]
Second objective (makespan): 23.1


In [82]:
M.fintime

[20.0, 20.299999999999997, 16.7, 17.5, 19.4, 19.0, 18.3, 19.9, 23.1]

In [83]:
# 類似Gantt Chart
# () is 1-indexed job name, the attached value is its associated proc time
# idle is the gap appears to preserve precedence between the operations within the same job 
print(*M.schedule, sep = '\n')

['(5, 1) 6.40', '(19, 1) 3.10', '(19, 2) 0.50', '(8, 1) 5.50', '(7, 2) 0.20', '(10, 2) 2.60', 'idle 1.10', '(15, 2) 0.60']
['(20, 1) 6.80', '(20, 2) 6.60', 'idle 6.50', '(9, 2) 0.40']
['(2, 1) 9.60', '(16, 1) 6.20', '(8, 2) 0.90']
['(3, 1) 4.80', '(6, 1) 3.50', '(13, 2) 4.30', '(1, 1) 4.90']
['(7, 1) 9.60', '(6, 2) 5.00', '(15, 1) 4.80']
['(13, 1) 7.60', '(5, 2) 1.00', '(11, 1) 3.50', '(17, 2) 6.90']
['(17, 1) 9.00', '(14, 1) 8.70', '(16, 2) 0.60']
['(18, 1) 5.30', '(18, 2) 1.70', '(12, 1) 0.50', '(12, 2) 5.80', '(9, 1) 6.60']
['(4, 1) 8.90', '(3, 2) 1.20', '(10, 1) 5.40', '(11, 2) 2.60', '(1, 2) 5.00']


## Testing all instances

In [87]:
Results = []
for instidx in range(5):
    data = instances[instidx]
    M = Machines(data)
    J = Jobs(len(data))
    J.add_jobs(data)
    heuristic(J = J, M = M)
    print(f'** Summary \ninstance {instidx}:')
    Tardy_jobs = list(np.where(J.completion_times > J.due_dates)[0])
    Tardy_jobs = [x+1 for x in Tardy_jobs]
    Makespan = max(M.fintime)
    print('First objective (# tardy):', len(Tardy_jobs), Tardy_jobs)
    print('Second objective (makespan):', Makespan)
    print('==================================')
    Results.append({'J':J, 'M':M})

[INFO] 12 jobs, 5 machines
[INFO] Tolerance: 1.81
** Summary 
instance 0:
First objective (# tardy): 0 []
Second objective (makespan): 8.2
[INFO] 11 jobs, 5 machines
[INFO] Tolerance: 2.21
** Summary 
instance 1:
First objective (# tardy): 1 [7]
Second objective (makespan): 8.7
[INFO] 10 jobs, 5 machines
[INFO] Tolerance: 2.33
[INFO] Job 3 will be tardy even if scheduled, queue last.
[INFO] Job 3 will be tardy even if scheduled, queue last.
** Summary 
instance 2:
First objective (# tardy): 2 [3, 5]
Second objective (makespan): 10.2
[INFO] 15 jobs, 7 machines
[INFO] Tolerance: 5.06
[INFO] Job 2 will be tardy even if scheduled, queue last.
[INFO] Job 10 will be tardy even if scheduled, queue last.
[INFO] Job 3 will be tardy even if scheduled, queue last.
[INFO] Job 12 will be tardy even if scheduled, queue last.
[INFO] Job 4 will be tardy even if scheduled, queue last.
[INFO] Job 14 will be tardy even if scheduled, queue last.
[INFO] Job 7 will be tardy even if scheduled, queue last.
[I

In [85]:
print(*Results[2]['M'].schedule, sep = '\n')

['(3, 1) 3.00', '(2, 2) 2.00', 'idle 0.40', '(9, 2) 2.20']
['(2, 1) 3.00', '(5, 2) 2.50', '(8, 1) 1.30', '(3, 2) 3.40']
['(1, 1) 3.00', '(1, 2) 1.50', '(10, 1) 2.60']
['(5, 1) 1.20', '(6, 1) 3.30', '(7, 1) 1.70', 'idle 0.90', '(10, 2) 1.30']
['(4, 1) 3.00', '(4, 2) 0.60', '(9, 1) 1.80', 'idle 0.80', '(7, 2) 1.50']


In [86]:
print(*Results[2]['J'].due_dates, sep = ', ')
print('====')
print(*Results[2]['J'].completion_times, sep = ', ')

5, 5, 5, 5, 5, 5, 10, 10, 10, 10
====
4.5, 5.0, 10.2, 3.6, 5.5, 4.5, 7.7, 6.8, 7.6000000000000005, 8.4


## Output

1. Write your heuristic algorithm here.
2. We would call this function in CA2_grading_program.py to evaluate your algorithm.
3. Please do not change the function name and the file name.
4. The parameter is the file path of a data file, whose format is specified in the document. 
5. You need to return your schedule in two lists "machine" and "completion_time".
    (a) machine[j][0] is the machine ID of the machine to process the first stage of job j + 1, and 
machine[j][1] is the machine to process the second stage of job j + 1.
    (b) completion_time[j][0] is the completion time of the first stage of job j + 1, and 
completion_time[j][1] is the completion time of the second stage of job j + 1. 
    
    
    Note 1. If you have n jobs, both the two lists are n by 2 (n rows, 2 columns). 
    Note 2. In the list "machine", you should record the IDs of machines 
            (i.e., to let machine 1 process the first stage of job 1, 
            you should have machine[0][0] == 1 rather than machine[0][0] == 0).
6. You only need to submit this algorithm_module.py.

In [100]:
def make_result(J):
    '''pass in the resulted J'''
    n = len(J.jobs)
    machine, completion_time = [], []
    for i in range(n):
        op1_mach_id = J.jobs[i].assign_mach[0]+1
        op2_mach_id = J.jobs[i].assign_mach[1]+1
        op1_c_time = round(J.jobs[i].end_time[0], 3)
        op2_c_time = round(J.jobs[i].end_time[1], 3)
        machine.append([op1_mach_id, op2_mach_id])
        completion_time.append([op1_c_time, op2_c_time])
    assert len(machine) == len(completion_time) == n
    return machine, completion_time

In [101]:
make_result(Results[0]['J'])

([[1, 5],
  [3, 3],
  [4, 2],
  [2, 1],
  [3, 4],
  [5, 0],
  [2, 4],
  [3, 3],
  [4, 5],
  [3, 1],
  [2, 4],
  [5, 0]],
 [[2.7, 4.0],
  [1.6, 3.8],
  [0.7, 3.3],
  [3.8, 4.5],
  [2.4, 4.4],
  [2.5, 2.5],
  [1.4, 3.4],
  [4.9, 7.0],
  [5.2, 6.7],
  [5.9, 6.4],
  [6.8, 8.2],
  [6.0, 6.0]])